In [ ]:
############# Results in pickle from csv files ###################
path = "C:\\ASM\\DevData\\eating\\python\\results\\"
res = []
for subject in range(11):
    res_lab = [];
    if subject<=4:
        sess_count = 2
        if subject<=1:
            sess_count = 1
        for sess in range(sess_count):
            res_sess = np.genfromtxt(path+"lab_"+str(subject)+"_"+str(sess)+".csv", delimiter=",")
            print(res_sess.shape)
            res_lab.append(res_sess)
    
    res_free = []
    sess_count = 2
    if subject==9:
        sess_count = 5
    for sess in range(sess_count):
        res_sess = np.genfromtxt(path+"free_"+str(subject)+"_"+str(sess)+".csv", delimiter=",")
        res_free.append(res_sess)
        
    res_subject = {"lab":res_lab, "free":res_free}    
    res.append(res_subject)
    
path = "C:\\ASM\\DevData\\eating\\python\\results\\res.pkl"
with open(path, "wb") as file:
    pickle.dump(res, file)

In [ ]:
#x = X_train[5, :, :]
#x = np.sqrt(np.sum(np.multiply(x, x), axis = 1))
#print(x)

m = mps[0][0][0]
s = segs[0][0][0]["x"]

print(len(m), ",", np.shape(s))
x = s[0, :10, :]
print(x)
x = np.sqrt(np.sum(np.multiply(x, x), axis = 1))
print(x)

In [ ]:
def find_clusters_dbscan(bites, eps, minPts):    
    count = len(bites)
    print("Detected bite count: ", count)
    if count == 0:
        return []
    
    pt = np.zeros((count, 2))
    
    for i in range(count):
        ct = 0
        j = i-1
        while j>=0 and bites[i] - bites[j]<=eps:
            ct+=1
            j-=1
            
        j = i+1
        while j<count and bites[j] - bites[i]<=eps:
            ct+=1
            j+=1
        
        pt[i, 1] = ct
        if ct >= minPts:
            pt[i, 0] = 1
    
    clusters = []
    six = -1
    eix = -1
    for i in range(count):        
        if pt[i, 0] == 1 and six < 0:            
            j = i-1
            while j>=0 and bites[i] - bites[j]<=eps:                
                j-=1
            six = j+1
            
        elif pt[i, 0] != 1 and six >= 0:                        
            j = i+1
            while j<count and bites[j] - bites[i]<=eps:                
                j+=1
            eix = j-1
            clusters.append([six, eix])
            six = -1
            eix = -1
            
    return clusters

In [ ]:
def find_meals_dbscan(mp, res):
    eps = 1*60*16 # 1 minutes
    minPts = 3
    bites = mp[res[:, 1]==1, 0]
    #print(bites*0.0625)
    
    clusters = find_clusters_dbscan(bites, eps, minPts)
    if len(clusters) ==0:
        return []
    
    clusters = np.array(clusters)
    meals = np.zeros((len(clusters), 3))
    for i in range(len(clusters)):
        six = clusters[i, 0]
        eix = clusters[i, 1]
        meals[i, 0] = bites[six]
        meals[i, 1] = bites[eix]
        meals[i, 2] = eix-six+1
    return meals